# Transform Existing Code to FL Easily with the FLARE Client API

The FLARE Client API provides an easy way to convert centralized, local training code into federated learning code with just a few lines of code changes.

Most of the previous examples up this point have already been using the Client API, but in this section we focus on the core concepts of the Client API and explain some of the ways it can be configured to help you use the Client API more effectively.

You can see the detailed examples with actual integration with deep learing platforms including PyTorch and TensorFlow here: https://github.com/NVIDIA/NVFlare/tree/main/examples/hello-world/ml-to-fl

## Core Concept

The general structure of the popular federated learning (FL) workflow, "FedAvg" is as follows:

1. **FL server initializes an initial model**
2. **For each round (global iteration):**
    1. FL server sends the global model to clients
    2. Each FL client starts with this global model and trains on their own data
    3. Each FL client sends back their trained model
    4. FL server aggregates all the models and produces a new global model

On the client side, the training workflow is as follows:

1. Receive the model from the FL server
2. Perform local training on the received global model and/or evaluate the received global model for model selection
3. Send the new model back to the FL server

To convert a centralized training code to federated learning, we need to
adapt the code to do the following steps:

1. Obtain the required information from the received `fl_model`
2. Run local training
3. Put the results in a new `fl_model` to be sent back

For a general use case, there are three essential methods for the Client API:

* ``init()``: Initializes NVFlare Client API environment.
* ``receive()``: Receives model from NVFlare side.
* ``send()``: Sends the model to NVFlare side.

You can use the Client API to change centralized training code to
federated learning, for example:

In [ ]:
import nvflare.client as flare

flare.init() # 1. Initializes NVFlare Client API environment.
input_model = flare.receive() # 2. Receives model from NVFlare side.
params = input_model.params # 3. Obtain the required information from received FLModel

# original local training code begins
new_params = local_train(params)
# original local training code ends

output_model = flare.FLModel(params=new_params) # 4. Put the results in a new FLModel
flare.send(output_model) # 5. Sends the model to NVFlare side.

With 5 lines of code changes, we convert the centralized training code to work in a
federated learning setting.

After this, we can use the job templates and the Job CLI
to generate a job and export it to run on a deployed NVFlare system or directly run the job using FL Simulator.

To see a table of the key Client APIs, see the [Client API documentation in the programming guide](https://nvflare.readthedocs.io/en/main/programming_guide/execution_api_type/client_api.html#id2).

Please consult the [Client API Module](https://nvflare.readthedocs.io/en/2.4/apidocs/nvflare.client.api.html) for more in-depth information about all of the Client API functions.

If you are using PyTorch Lightning in your training code, you can check the [Lightning API Module](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.app_opt.lightning.api.html). Also, be sure to look through the [Convert Torch Lightning to FL notebook](../02.2_convert_torch_lightning_to_federated_learning/convert_torch_lightning_to_fl.ipynb) and related code.

## Advanced User Options: Client API with Different Implementations

Within the Client API, we offer multiple implementations tailored to diverse requirements:

* In-process Client API: In this setup, the client training script operates within the same process as the NVFlare Client job.
This configuration, utilizing the ```InProcessClientAPIExecutor```, offers shared memory usage and is efficient with simple configuration. 
This is the default for `ScriptRunner` since by default `launch_external_process=False`. Use this configuration for development or single GPU training.

* Sub-process Client API: Here, the client training script runs in a separate subprocess.
Utilizing the ```ClientAPILauncherExecutor```, this option offers flexibility in communication mechanisms:
  * Communication via CellPipe (default)
  * Communication via FilePipe (no capability to stream metrics for experiment tracking) 
This configuration is ideal for scenarios requiring multi-GPU or distributed PyTorch training.

Choose the option best suited to your specific requirements and workflow preferences.

These implementations can be easily configured using the JobAPI's `ScriptRunner`.
By default, the ```InProcessClientAPIExecutor``` is used, however setting `launch_external_process=True` uses the ```ClientAPILauncherExecutor```
with pre-configured CellPipes for communication and metrics streaming.